In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import pandas as pd
import imgaug as ia
ia.seed(1)
from imgaug import augmenters as iaa 
import imageio
import os, json, cv2, random
from google.colab.patches import cv2_imshow
from pathlib import Path
from google.colab import drive

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader 
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

In [ ]:
import pandas as pd
import os

In [ ]:
images_dir = Path("drive/MyDrive/Hackathon-Challenge-1-Train/")
annotations_csv_f = Path("drive/MyDrive/train-challenge-2-first-attempt.csv")

In [ ]:
from detectron2.structures import BoxMode

class Annotations:
  def __init__(self):
    self.annotations = None
    self.categories = None
    self.categories_map_reverse = None

  def compute(self):
    if self.annotations != None:
      return self.annotations

    # first read all the information
    csv_data = {}
    with annotations_csv_f.open() as f:
      for line in f:
        image_name, xmin, ymin, xmax, ymax, label = line.rstrip().split(",")
        if image_name not in csv_data:
          csv_data[image_name] = []
        csv_data[image_name].append([xmin,ymin,xmax,ymax,label])
    print(f"Finished reading data. Images: {len(csv_data.keys())}")

    # get all the labels and sort them (use a dictionary for O(1) lookup)
    categories = []
    for anns in csv_data.values():
      for ann in anns:
        label = ann[4]
        if not label in categories:
          categories.append(label)
    categories = sorted(categories)
    print(f"First 10 categories are {categories[:10]}")
    self.categories = categories
    categories_map = {label:idx for idx,label in enumerate(categories)}
    self.categories_map_reverse = {idx:label for idx,label in enumerate(categories)}

    # convert this to the format that detectron2 wants
    data = []
    for idx, image_name in enumerate(csv_data.keys()):

      if idx >= 2000: # load more if you want!
        break

      if idx%100 == 0:
        print(f"Processed {idx} images.")
      record = {}

      filename = os.path.join(images_dir,image_name)  #str(images_dir/image_name)
      height, width = cv2.imread(filename).shape[:2]
      #imageio.imwrite(filename,image_augmentations)
      
      record["file_name"] = filename
      record["image_id"] = idx
      record["height"] = height
      record["width"] = width

      annotations = csv_data[image_name]
      objs = []
      for annotation in annotations:
        xmin, ymin, xmax, ymax, label = annotation
        objs.append({
            "bbox": [float(xmin),float(ymin),float(xmax),float(ymax)],
            "bbox_mode": BoxMode.XYXY_ABS,
            "category_id": categories_map[label]
        })
      record["annotations"] = objs
      data.append(record)
    
    self.annotations = data

anns = Annotations()
anns.compute()
print(anns.categories_map_reverse)
print(f"Finished loading annotations.")

In [ ]:
# let's register this dataset (unregistered if it exists.)
def register(name, load_fn, labels):
  if name in DatasetCatalog.list():
    DatasetCatalog.remove(name)
  if name in MetadataCatalog.list():
    MetadataCatalog.remove(name)
  
  DatasetCatalog.register(name, load_fn)
  MetadataCatalog.get(name).set(thing_classes=labels)

register("train", lambda:anns.annotations, anns.categories)
#register("validation",lambda:anns_val.annotations, anns_val.categories)
print("Dataset registered.")

In [ ]:
for d in random.sample(anns.annotations, 1):
  img = cv2.imread(d["file_name"])
  visualizer = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get("train"), scale=0.5)
  out = visualizer.draw_dataset_dict(d)
  cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")) # COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml
#COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo 
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  
cfg.SOLVER.MAX_ITER = 9000    
cfg.SOLVER.STEPS = []      
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(anns.categories) 

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the trained model
predictor = DefaultPredictor(cfg)

In [ ]:
# visualize inference on some images
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0 # if you want to allow predictions with lower accuracy
predictor = DefaultPredictor(cfg)
for d in random.sample(DatasetCatalog.get(test_dataset_name), 1):
  img = cv2.imread(d["file_name"])
  outputs = predictor(img)
  visualizer = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get(test_dataset_name), scale=1)
  # out = visualizer.draw_dataset_dict(d) # these are the ground truths 
  out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
images_test_dir = 'drive/MyDrive/EdIntelligence-Neurolabs-Hackathon/challenge-2/test/images/'

In [ ]:
outputs = []
file_names = []
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0
for filename in os.listdir(images_test_dir):
  image = cv2.imread(os.path.join(images_test_dir,filename))
  outputs.append(predictor(image))
  file_names.append(filename)
print(len(outputs))

In [ ]:
print(file_names)
print((outputs[0]["instances"].pred_classes.cpu().numpy()))
array = outputs[0]["instances"].pred_boxes.tensor.cpu().numpy()
array = np.array(array,dtype=np.int32)[0]
print(array)

In [ ]:
all_lists = []
for i in range(len(outputs)):
  bb_array = outputs[i]["instances"].pred_boxes.tensor.cpu().numpy()
  bb_array = np.array(bb_array,dtype=np.int32)
  for idx in range(len(outputs[i]["instances"].pred_classes.cpu().numpy())):
    all_lists.append([file_names[i],bb_array[idx][0],bb_array[idx][1],bb_array[idx][2],bb_array[idx][3],outputs[i]["instances"].pred_classes.cpu().numpy()[idx]])
df = pd.DataFrame(all_lists,columns=["ImageId","xmin","ymin","xmax","ymax","ClassLabels"])
df.head(100)


In [ ]:
df["ClassLabels"] = df["ClassLabels"].apply(lambda x: anns.categories_map_reverse[x] )
df

In [ ]:
df.to_csv("drive/MyDrive/submission-1-challenge-2-faster_rcnn_X_101_32x8d_FPN_3x-no-augment.csv",header=False,index=False)